In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)

## Read dataset

In [2]:
hpsc_county = pd.read_csv("COVID-19_HPSC_County_Statistics_Historic_Data.csv")
hse_vaccine = pd.read_csv("COVID-19_HSE_Weekly_Vaccination_Figures.csv")

## Task 3 

### Create Knowledge Graphs from county statistics

In [5]:
#Task 3 - Write a Python script using RDFlib (or similar) to transform the data into RDF.
# consider http://ex.org/ as URI prefix

from rdflib import Graph, plugin
from SPARQLWrapper import SPARQLWrapper
from rdflib.serializer import Serializer
from rdflib import URIRef, BNode, Literal, XSD
from rdflib.namespace import RDF, FOAF, ORG
from datetime import datetime

task3_rdf = Graph()

def addCounty(task3_rdf, subject, county_name):
    #add county name
    hasCountyName = URIRef("http://ex.org/county/hasCountyName")
    county_name = Literal(county_name)
    task3_rdf.add((subject, hasCountyName, county_name))
    
    return task3_rdf

def addCountyPop(task3_rdf, subject, population):
    #add population
    hasPopulation = URIRef("http://ex.org/county/hasPopulation")
    population = Literal(population,datatype=XSD.integer)
    task3_rdf.add((subject, hasPopulation, population))
    
    return task3_rdf

def addCountyLocation(task3_rdf, subject, IGEasting, IGNorthing, Lat, Long):
    #add location (IGEasting / IGNorthing / Lat / Long)
    locateAtIGEasting = URIRef("http://ex.org/county/locateAtIGEasting")
    IGEasting = Literal(IGEasting,datatype=XSD.integer)
    task3_rdf.add((subject, locateAtIGEasting, IGEasting))
    
    locateAtIGNorthing = URIRef("http://ex.org/county/locateAtIGNorthing")
    IGNorthing = Literal(IGNorthing,datatype=XSD.integer)
    task3_rdf.add((subject, locateAtIGNorthing, IGNorthing))
    
    locateAtLat = URIRef("http://ex.org/county/locateAtLat")
    Lat = Literal(Lat,datatype=XSD.float)
    task3_rdf.add((subject, locateAtLat, Lat))
    
    locateAtLong = URIRef("http://ex.org/county/locateAtLong")
    Long = Literal(Long,datatype=XSD.float)
    task3_rdf.add((subject, locateAtLong, Long))

    return task3_rdf

def addCountyShape(task3_rdf, subject, SHAPE_Length, SHAPE_Area):
    #add shape (SHAPE_Length / SHAPE_Area)
    hasShapeLength = URIRef("http://ex.org/county/hasShapeLength")
    SHAPE_Length = Literal(SHAPE_Length,datatype=XSD.float)
    task3_rdf.add((subject, hasShapeLength, SHAPE_Length))
    
    hasShapeArea = URIRef("http://ex.org/county/hasShapeArea")
    SHAPE_Area = Literal(SHAPE_Area,datatype=XSD.float)
    task3_rdf.add((subject, hasShapeArea, SHAPE_Area))

    return task3_rdf

def addCountyUGI(task3_rdf, subject, UGI):
    #add ugi
    representsAs = URIRef("http://ex.org/county/representsAs")
    UGI = Literal(UGI,datatype=XSD.anyURI)
    task3_rdf.add((subject, representsAs, UGI))
    
    return task3_rdf

def addCountyCovidRecord(task3_rdf, subject, object_):
    #add covid record (Class)
    hasCovidRecord = URIRef("http://ex.org/county/hasCovidRecord")
    task3_rdf.add((subject, hasCovidRecord, object_))
    
    return task3_rdf

def addCovidRecordStats(task3_rdf, subject, ConfirmedCovidCases, PopulationProportionCovidCases, ConfirmedCovidDeaths, ConfirmedCovidRecovered):
    #add covid stats to covid record
    hasConfirmedCovidCases = URIRef("http://ex.org/covidRecord/hasCovidRecord")
    ConfirmedCovidCases = Literal(ConfirmedCovidCases,datatype=XSD.integer)
    task3_rdf.add((covidRecord, hasConfirmedCovidCases, ConfirmedCovidCases))
    
    hasPopulationProportionCovidCases = URIRef("http://ex.org/covidRecord/hasPopulationProportionCovidCases")
    PopulationProportionCovidCases = Literal(PopulationProportionCovidCases,datatype=XSD.float)
    task3_rdf.add((covidRecord, hasPopulationProportionCovidCases, PopulationProportionCovidCases))
    
    hasConfirmedCovidDeaths = URIRef("http://ex.org/covidRecord/hasConfirmedCovidDeaths")
    ConfirmedCovidDeaths = Literal(ConfirmedCovidDeaths,datatype=XSD.float)
    task3_rdf.add((covidRecord, hasConfirmedCovidDeaths, ConfirmedCovidDeaths))
    
    hasConfirmedCovidRecovered = URIRef("http://ex.org/covidRecord/hasConfirmedCovidRecovered")
    ConfirmedCovidRecovered = Literal(ConfirmedCovidRecovered,datatype=XSD.float)
    task3_rdf.add((covidRecord, hasConfirmedCovidRecovered, ConfirmedCovidRecovered))
    
    return task3_rdf

def addCovidRecordTimestamp(task3_rdf, subject, TimeStamp): 
    #add timestamp to covid record
    recordOn = URIRef("http://ex.org/covidRecord/recordOn")
    TimeStamp = Literal(datetime.strptime(TimeStamp, '%Y/%m/%d 00:00:00+00').date(),datatype=XSD.date)
    task3_rdf.add((covidRecord, recordOn, TimeStamp))
    
    return task3_rdf
    
    
# loop through each line of dataset,
# define subject, predicate and object then add them to graph
for index, row in hpsc_county.iterrows():
    county = URIRef("http://ex.org/county/"+row['CountyName'])

    task3_rdf = addCounty(task3_rdf, county, row['CountyName'])
    task3_rdf = addCountyPop(task3_rdf, county, row['PopulationCensus16'])
    task3_rdf = addCountyLocation(task3_rdf, county, row['IGEasting'], row['IGNorthing'], row['Lat'], row['Long'])
    task3_rdf = addCountyShape(task3_rdf, county, row['SHAPE_Length'], row['SHAPE_Area'])
    task3_rdf = addCountyUGI(task3_rdf, county, row['UGI'])
    
    covidRecord = URIRef("http://ex.org/covidRecord/"+str(row['OBJECTID']))
    task3_rdf = addCountyCovidRecord(task3_rdf, county, covidRecord)
    task3_rdf = addCovidRecordStats(task3_rdf, covidRecord, 
                        row['ConfirmedCovidCases'], 
                        row['PopulationProportionCovidCases'],
                        row['ConfirmedCovidDeaths'],
                        row['ConfirmedCovidRecovered'])
    task3_rdf = addCovidRecordTimestamp(task3_rdf, covidRecord, row['TimeStamp'])
    
#     if index == 1:
#         break
    


## Task 4 

### Query from Wikidata to generate connection between Task 3 and Task 4

County and City of the vaccination centre is obtained from Wikidata. Then they are added to the vaccine dataset as new columns.

In [6]:
#Task 4 - Write a Python script using RDFlib (or similar) to transform the data into RDF.
#connect this data with the county statistics with location from wikidata

from qwikidata.sparql  import return_sparql_query_results
import numpy as np

unique_xypoints = set(zip(list(hse_vaccine['X']), list(hse_vaccine['Y'])))

wikidata_results = {}

for xy in unique_xypoints:
    wikidata_query = '''
        SELECT ?place ?location ?countyLabel ?distance ?placeLabel WHERE {
           ?place wdt:P7959 ?county .
            SERVICE wikibase:around { 
              ?place wdt:P625 ?location .
              bd:serviceParam wikibase:center "Point('''+f'''{xy[0]} {xy[1]}'''+''')"^^geo:wktLiteral . 
              bd:serviceParam wikibase:radius "0.5" . 
              bd:serviceParam wikibase:distance ?distance .

            } 
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }

        } ORDER BY ?distance LIMIT 1
    '''

    xypoints_results = return_sparql_query_results(wikidata_query)

    for row in xypoints_results['results']['bindings']:
        wikidata_results[xy] = {}
        wikidata_results[xy]['County'] = row['countyLabel']['value'].replace("County ", "")
        wikidata_results[xy]['City'] = row['placeLabel']['value']

wikidata_results

# add county and city column to HSE dataset, add triples to RDF with county as class, city as another class,
# vaccination info is properties under city
hse_vaccine['XY'] = list(zip(list(hse_vaccine['X']), list(hse_vaccine['Y'])))

county = {k: v['County'] for k,v in wikidata_results.items()}
city = {k: v['City'] for k,v in wikidata_results.items()}

county_city = {v['County']: v['City'] for k,v in wikidata_results.items()}

    
hse_vaccine['CountyName'] = hse_vaccine['XY'].map(county)
hse_vaccine['CityName'] = hse_vaccine['XY'].map(city)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "



### Create Knowledge Graphs from Vaccine dataset

In [7]:
#Task 4 - Write a Python script using RDFlib (or similar) to transform the data into RDF.
# consider http://ex.org/ as URI prefix

from rdflib import Graph, plugin
from SPARQLWrapper import SPARQLWrapper
from rdflib.serializer import Serializer
from rdflib import URIRef, BNode, Literal, XSD
from rdflib.namespace import RDF, FOAF, ORG
from datetime import datetime

task4_rdf = Graph()

def addCity(rdf, subject, city_name):
    #add city and city name
    hasCity = URIRef("http://ex.org/county/hasCity")
    city = URIRef(f"http://ex.org/city/{city_name}")
    rdf.add((subject, hasCity, city))
    
    hasCityName = URIRef("http://ex.org/city/hasCityName")
    city_name = Literal(city_name)
    rdf.add((city, hasCityName, city_name))
    
    return rdf
    
def addVaccinationCentre(task3_rdf, subject, vaccinationCentre):
    #add vaccinationCentre
    hasVaccinationCentre = URIRef("http://ex.org/city/hasVaccinationCentre")
    task3_rdf.add((subject, hasVaccinationCentre, vaccinationCentre))
    
    return task3_rdf

def addVCLocation(task3_rdf, subject, X, Y):
    #add vc location
    hasXPoint = URIRef("http://ex.org/vaccinationCentre/hasXPoint")
    X = Literal(X,datatype=XSD.float)
    task3_rdf.add((subject, hasXPoint, X))
    
    hasYPoint = URIRef("http://ex.org/vaccinationCentre/hasYPoint")
    Y = Literal(Y,datatype=XSD.float)
    task3_rdf.add((subject, hasYPoint, Y))
    
    return task3_rdf

def addVaccinationRecord(rdf, subject, object_):
    #add vcRecord
    hasVaccinationRecord = URIRef("http://ex.org/vaccinationCentre/hasVaccinationRecord")
    rdf.add((subject, hasVaccinationRecord, object_))
    
    return rdf

def addExtractDate(task3_rdf, subject, TimeStamp):
    #add date to vaccination record
    extractOn = URIRef("http://ex.org/vaccinationRecord/extractOn")
    TimeStamp = Literal(datetime.strptime(TimeStamp, '%Y/%m/%d').date(),datatype=XSD.date)
    task3_rdf.add((subject, extractOn, TimeStamp))
    
    return task3_rdf

def addWeek(task3_rdf, subject, week):
    #add week
    recordOn = URIRef("http://ex.org/vaccinationRecord/recordOn")
    week = Literal(week)
    task3_rdf.add((subject, recordOn, week))

    return task3_rdf

def addTotalweeklyVaccines(task3_rdf, subject, TotalWeeklyVaccines):
    #add TotalWeeklyVaccines
    hasTotalWeeklyVaccines = URIRef("http://ex.org/vaccinationRecord/hasTotalWeeklyVaccines")
    TotalWeeklyVaccines = Literal(TotalWeeklyVaccines,datatype=XSD.integer)
    task3_rdf.add((subject, hasTotalWeeklyVaccines, TotalWeeklyVaccines))
    
    return task3_rdf

def addGenderVaccination(task3_rdf, subject, id_, male, female, na):
    #add female, male, na weekly vaccinated value
    hasGender = URIRef("http://ex.org/vaccinationRecord/hasGender")
    male_gender = URIRef(f"http://ex.org/{id_}_Male")
    task3_rdf.add((subject, hasGender, male_gender))
    female_gender = URIRef(f"http://ex.org/{id_}_Female")
    task3_rdf.add((subject, hasGender, female_gender))
    na_gender = URIRef(f"http://ex.org/{id_}_NA")
    task3_rdf.add((subject, hasGender, na_gender))
    
    hasName = URIRef("http://ex.org/gender/hasName")
    gender = Literal("Male")
    task3_rdf.add((male_gender, hasName, gender))
    gender = Literal("Female")
    task3_rdf.add((female_gender, hasName, gender))
    gender = Literal("NA")
    task3_rdf.add((na_gender, hasName, gender))
    
    hasValue = URIRef("http://ex.org/vaccinationRecord/hasValue")
    male = Literal(male,datatype=XSD.integer)
    task3_rdf.add((male_gender, hasValue, male))
#     hasFemaleValue = URIRef("http://ex.org/vaccinationRecord/hasFemaleValue")
    female = Literal(female,datatype=XSD.integer)
    task3_rdf.add((female_gender, hasValue, female))
#     hasNAValue = URIRef("http://ex.org/vaccinationRecord/hasNAValue")
    na = Literal(na,datatype=XSD.integer)
    task3_rdf.add((na_gender, hasValue, na))
    
    return task3_rdf

def addVaccineTypeVaccination(task3_rdf, subject, id_, moderna, pfizer, janssen, az):
    #add vaccines type to vaccination record
    hasVaccine = URIRef("http://ex.org/vaccinationRecord/hasVaccine")
    moderna_vaccine = URIRef(f"http://ex.org/{id_}_Moderna")
    task3_rdf.add((subject, hasVaccine, moderna_vaccine))
    pfizer_vaccine = URIRef(f"http://ex.org/{id_}_Pfizer")
    task3_rdf.add((subject, hasVaccine, pfizer_vaccine))
    janssen_vaccine = URIRef(f"http://ex.org/{id_}_Janssen")
    task3_rdf.add((subject, hasVaccine, janssen_vaccine))
    az_vaccine = URIRef(f"http://ex.org/{id_}_AstraZeneca")
    task3_rdf.add((subject, hasVaccine, az_vaccine))
    
    hasName = URIRef("http://ex.org/vaccine/hasName")
    vaccine = Literal("Moderna")
    task3_rdf.add((moderna_vaccine, hasName, vaccine))
    vaccine = Literal("Pfizer")
    task3_rdf.add((pfizer_vaccine, hasName, vaccine))
    vaccine = Literal("Janssen")
    task3_rdf.add((janssen_vaccine, hasName, vaccine))
    vaccine = Literal("AstraZeneca")
    task3_rdf.add((az_vaccine, hasName, vaccine))
    
    #add weekly value of each vaccines type to vaccination type
    hasValue = URIRef("http://ex.org/vaccinationRecord/hasValue")
    moderna = Literal(moderna,datatype=XSD.integer)
    task3_rdf.add((moderna_vaccine, hasValue, moderna))
    pfizer = Literal(pfizer,datatype=XSD.integer)
    task3_rdf.add((pfizer_vaccine, hasValue, pfizer))
    janssen = Literal(janssen,datatype=XSD.integer)
    task3_rdf.add((janssen_vaccine, hasValue, janssen))
    az = Literal(az,datatype=XSD.integer)
    task3_rdf.add((az_vaccine, hasValue, az))
    
    return task3_rdf

def addAgeGroupVaccination(task3_rdf, subject, id_, value, column, agegroup): 
    #add age group to vaccination record
    hasAgeGroup = URIRef(f"http://ex.org/vaccinationRecord/hasAgeGroup")
    ageGroup = URIRef(f"http://ex.org/{id_}_{agegroup.replace('_', '')}")
    task3_rdf.add((subject, hasAgeGroup, ageGroup))
    
#     status = 'Fully'
#     if "Par" in column: status = 'Partially'
#     hasStatus = URIRef(f"http://ex.org/hasVaccinationStatus")
#     vaccinationStatus = URIRef(f"http://ex.org/{status}")
#     task3_rdf.add((subject, hasStatus, vaccinationStatus))
    
    #add age group value
    hasValue = URIRef(f"http://ex.org/vaccinationRecord/has{column.replace('_', '')}Value")
    if 'Partial_' in column:
        value = Literal(value,datatype=XSD.integer)
    else:
        value = Literal(value,datatype=XSD.float)
    task3_rdf.add((ageGroup, hasValue, value))
    
    return task3_rdf
    
    
# loop through each line of dataset,
# define subject, predicate and object then add them to graph
for k, v in county_city.items():
    county = URIRef("http://ex.org/county/"+k)
    task4_rdf = addCity(task4_rdf, county, v)
    
AgeGroup = ['0to9', '10to19', '20to29', '30to39', '40to49', '50to59', 
           '60to69', '70to79', '80_', 'NA']
ColumnName = ['Partial_', 'ParCum_', 'ParPer_',
             'Fully_', 'FullyCum_', 'FullyPer_']
withAge80_ = ['Partial_', 'Fully_']
for index, row in hse_vaccine.iterrows():    
    city = URIRef("http://ex.org/city/"+row['CityName'])
    
    vaccinationCentre = URIRef("http://ex.org/vaccinationCentre/"+str(row['X'])+"_"+str(row['Y']))
    task4_rdf = addVaccinationCentre(task4_rdf, city, vaccinationCentre)
    task4_rdf = addVCLocation(task4_rdf, vaccinationCentre, row['X'], row['Y'])
    
    vaccinationRecord = URIRef("http://ex.org/vaccinationRecord/"+str(row['ObjectId']))
    
    task4_rdf = addVaccinationRecord(task4_rdf, vaccinationCentre, vaccinationRecord)
    task4_rdf = addExtractDate(task4_rdf, vaccinationRecord, row['ExtractDate'])
    task4_rdf = addWeek(task4_rdf, vaccinationRecord, row['Week'])
    task4_rdf = addTotalweeklyVaccines(task4_rdf, vaccinationRecord, row['TotalweeklyVaccines'])
    
    task4_rdf = addGenderVaccination(task4_rdf, vaccinationRecord, row['ObjectId'],
                                     row['Male'], row['Female'], row['NA'])
    task4_rdf = addVaccineTypeVaccination(task4_rdf, vaccinationRecord, row['ObjectId'],
                                          row['Moderna'], row['Pfizer'], row['Janssen'], row['AstraZeneca'])
    for agegroup in AgeGroup:
        for column in ColumnName:
            if (agegroup == '80_' and column not in withAge80_) or (agegroup == 'NA'):
                task4_rdf = addAgeGroupVaccination(task4_rdf, vaccinationRecord, row['ObjectId'], row[f'{column}{agegroup}'], column, agegroup)
            else:
                task4_rdf = addAgeGroupVaccination(task4_rdf, vaccinationRecord, row['ObjectId'], row[f'{column}Age{agegroup}'], column, agegroup)
    
#     break
#     if index == 1:
#         break
    

In [8]:
# query = """
#     SELECT DISTINCT ?g ?v
#     WHERE {
#         ?a <http://ex.org/vaccinationCentre/hasVaccinationRecord> ?b .
#         ?b <http://ex.org/vaccinationRecord/hasAgeGroup> ?g . 
#         ?g <http://ex.org/vaccinationRecord/hasFullyCumValue> ?v .
#         #?g <http://ex.org/vaccine/hasName> ?l .
#         FILTER(?b = <http://ex.org/vaccinationRecord/10>)
#     }
# """

# res = task4_rdf.query(query)
# for row in res:
#     print(row.g, row.v)
# '''
# http://ex.org/10_0to9 2.0
# http://ex.org/10_10to19 300.0
# http://ex.org/10_20to29 18690.0
# http://ex.org/10_30to39 29775.0
# http://ex.org/10_40to49 36653.0
# http://ex.org/10_50to59 33248.0
# http://ex.org/10_60to69 18652.0
# http://ex.org/10_70to79 7093.0
# http://ex.org/10_80 17301.0
# http://ex.org/10_NA 17.0
# '''

## Merge Knowledge Graphs from Task 3 and Task 4

In [9]:
merged_rdf = task3_rdf + task4_rdf

## Task 5

Question Answering based on Knowledge Graphs

In [15]:
#Task 5 - Write SPARQL queries or equivalent Python functions that can answer the following questions:

# ● How many cases were there in Galway on 1st September 2021?

query = """
    SELECT DISTINCT ?date ?value
    WHERE {
        ?a <http://ex.org/county/hasCountyName> ?b .
        ?a <http://ex.org/county/hasCovidRecord> ?c .
        ?c <http://ex.org/covidRecord/recordOn> ?date .
        ?c <http://ex.org/covidRecord/hasCovidRecord> ?value . 
        FILTER(?b = 'Galway' && ?date = "2021-09-01"^^xsd:date)
    }
"""

res = merged_rdf.query(query)
for row in res:
    print(f"1. There where {row.value} covid cases in Galway on 1st September 2021.")
    
# ● How many cases were there in the month of September in all counties?

query = """
    SELECT DISTINCT ?b ?value
    WHERE {
        ?a <http://ex.org/county/hasCovidRecord> ?b .
        ?b <http://ex.org/covidRecord/recordOn> ?date .
        ?b <http://ex.org/covidRecord/hasCovidRecord> ?value . 
        FILTER(month(?date) = 9)
    }
"""

res = merged_rdf.query(query)
total_covidCases = 0
for row in res:
    total_covidCases += int(row.value)
print(f"2. There where {total_covidCases} of covid cases in the month of September in all counties.")

# ● In which week were there the most vaccinations?
query = """
    SELECT DISTINCT ?week
    WHERE {
        ?a <http://ex.org/vaccinationCentre/hasVaccinationRecord> ?b .
        ?b <http://ex.org/vaccinationRecord/recordOn> ?week .
        ?b <http://ex.org/vaccinationRecord/hasTotalWeeklyVaccines> ?value . 
    }
    ORDER BY DESC(?value)
    LIMIT 1
"""

res = merged_rdf.query(query)
for row in res:
    print(f"3. In week {row.week} there were the most vaccinations.")


1. There where 16175 covid cases in Galway on 1st September 2021.
2. There where 61959083 of covid cases in the month of September in all counties.
3. In week 2021-W27 there were the most vaccinations.


In [11]:
query = """
    SELECT DISTINCT ?countyLabel ?cityLabel ?week ?value
    WHERE {
        ?county <http://ex.org/county/hasCity> ?city .
        ?county <http://ex.org/county/hasCountyName> ?countyLabel .
        ?city <http://ex.org/city/hasCityName> ?cityLabel .
        
        ?city <http://ex.org/city/hasVaccinationCentre> ?a .
        
        ?a <http://ex.org/vaccinationCentre/hasVaccinationRecord> ?b .
        ?b <http://ex.org/vaccinationRecord/recordOn> ?week .
        ?b <http://ex.org/vaccinationRecord/hasTotalWeeklyVaccines> ?value . 
    }
    ORDER BY DESC(?value)
    LIMIT 1
"""

res = merged_rdf.query(query)
for row in res:
#     print(row.countyLabel, row.cityLabel, row.week, row.value)
    print(f"""Additional. The vaccination centre that has most vaccinations is located in {row.cityLabel}, {row.countyLabel}.
          """)

Additional. The vaccination centre that has most vaccinations is located in Pollagh, Offaly.
          
